In [39]:
import pandas as pd

In [40]:
bitcoin_prices_df = pd.read_parquet('../data/bitcoin_prices.parquet.gzip')
reddit_titles_df = pd.read_parquet('../data/reddit_titles.parquet.gzip')
sentiment_scores_df = pd.read_parquet('../data/sentiment_scores.parquet.gzip')

In [41]:
sentiment_scores_df = sentiment_scores_df.rename(columns={
    'date': 'orig_date',
    'offset_date': 'date'
})

In [43]:
pd.qcut(bitcoin_prices_df['price_diff'], q=num_buckets)

0                        NaN
1          (273.887, 2192.6]
2      (-2574.701, -194.023]
3        (-194.023, 273.887]
4      (-2574.701, -194.023]
               ...          
236    (-2574.701, -194.023]
237      (-194.023, 273.887]
238        (273.887, 2192.6]
239      (-194.023, 273.887]
240        (273.887, 2192.6]
Name: price_diff, Length: 241, dtype: category
Categories (3, interval[float64]): [(-2574.701, -194.023] < (-194.023, 273.887] < (273.887, 2192.6]]

In [29]:
num_buckets = 3
bitcoin_prices_df['price_diff'] = pd.qcut(bitcoin_prices_df['price_diff'], q=num_buckets, labels=range(num_buckets))

In [30]:
bitcoin_prices_df['price_diff_lag_1'] = bitcoin_prices_df['price_diff'].shift(1)
bitcoin_prices_df['price_diff_lag_2'] = bitcoin_prices_df['price_diff'].shift(2)
bitcoin_prices_df['price_diff_lag_3'] = bitcoin_prices_df['price_diff'].shift(3)

In [31]:
bitcoin_prices_df = bitcoin_prices_df.reset_index(drop=True)

In [32]:
bitcoin_prices_df['day'] = bitcoin_prices_df['date'].dt.day

In [33]:
bitcoin_prices_df['hour'] = bitcoin_prices_df['date'].dt.hour

In [34]:
everything = bitcoin_prices_df.join(sentiment_scores_df.set_index('date'), on='date')

In [35]:
everything = everything.dropna()

In [36]:
features_df = everything[['date', 'price_diff_lag_1', 'price_diff_lag_2', 'price_diff_lag_3', 'day', 'hour', 'compound']]
target_df = everything[['price_diff', 'date']]

In [37]:
everything = features_df.join(target_df.set_index('date'), on='date')

In [38]:
everything.to_parquet('../data/everything_v2.parquet.gzip', compression='gzip')